In [5]:
# model용 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataset as dt
import crawling as cl
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM,Bidirectional
from tensorflow.keras.models import Sequential
from keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflowjs as tfjs
from pandas.plotting import scatter_matrix


In [ ]:
# dataset용 improt
import pandas as pd
import numpy as np
from pykospacing import Spacing
from konlpy.tag import Komoran
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.normalizer import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
# text 전처리 class
class text_processing():
    def __init__(self,dataset):
        self.dataset = dataset
        self.token_size = 0

    def text_normalization(self):
        self.dataset = self.dataset.dropna(how = 'any')
        self.dataset['Title'] = self.dataset['Title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")
        self.dataset['Title'].replace('', np.nan, inplace=True)
        return self.dataset

    def text_tokenization(self):
        komoran = Komoran(userdic= 'dic.txt')
        temp = []
        for sentence in self.dataset['Title']:
            temp.append(komoran.nouns(sentence))
        self.dataset['Title'] = temp
        return self.dataset

    def text_integer(self):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.dataset['Title'])
        token_size = len(tokenizer.word_index)

        tokenizer = Tokenizer(token_size)
        tokenizer.fit_on_texts(self.dataset['Title'])

        self.dataset['Title'] = tokenizer.texts_to_sequences(self.dataset['Title'])
        self.token_size = token_size
        return self.dataset

In [3]:
def create_predict(dataset):
    dataset = pd.read_csv('dataset.csv', low_memory=False)
    y_train = np.array(dataset['Binary'])
    x_train = []
    for index in dataset['Title']:
        x_train.append(np.fromstring(index, dtype=int, sep=','))

    x_train = pad_sequences(x_train, maxlen = 12)

    # x_train, x_test, y_train, y_test = train_test_split(
    #     x_train, y_train, test_size=0.2, random_state=1)

    model = Sequential()

    # model.add(Embedding(1152, 100))
    # model.add(LSTM(128))
    # model.add(Dense(1, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # model.add(layers.Dense(16, activation='relu', input_shape=(len(dataset),)))
    # model.add(layers.Dense(16, activation='relu'))
    # model.add(layers.Dense(1, activation='sigmoid'))


    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    # history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=30, validation_split=0.2)
    history = model.fit(x_train, y_train, epochs=30, batch_size=60, validation_split=0.2)

    # loaded_model = load_model('best_model.h5')
    return model


def sentiment_text_processing():
    m_dataset = cl.single_page_crawling_for_modeling()
    print(m_dataset['Title'])
    m_dataset = dt.text_processing(m_dataset)
    m_dataset.text_normalization()
    m_dataset.text_tokenization()
    m_dataset.text_integer()

    return m_dataset.dataset
  
def sentiment_predict(m_dataset, model):
    x_train = pad_sequences(m_dataset['Title'], maxlen = 12)

    # for sentence in x_train:
    score = model.predict(x_train)

    print(score)

    for s in score:
        s = float(s)
        if(s > 0.5):
          print("{:.2f}% 확률로 비교과프로그램입니다.\n".format(s * 100))
        else:
          print("{:.2f}% 확률로 비교과 프로그램이 아닙니다.\n".format((1 - s) * 100))

def visualize(m_dataset,m_target):
    x_train = []
    for index in dataset['Title']:
        x_train.append(np.fromstring(index, dtype=int, sep=','))

In [1]:
from kafka import KafkaProducer
from json import dumps
import time

producer = KafkaProducer(acks=0, compression_type='gzip', bootstrap_servers=['localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8')) 
start = time.time() 
for i in range(10000): 
    data = {'str' : 'result'+str(i)} 
    producer.send('quickstart-events', value=data) 
    producer.flush()
print("elapsed :", time.time() - start)


ImportError: cannot import name 'KafkaProducer' from partially initialized module 'kafka' (most likely due to a circular import) (C:\Users\wkdgn\Desktop\Study\3-2\SelfDirectedProjectII\all-knu-ml\kafka.py)